In [8]:
import os
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 1) Launch headless Chrome
opts = Options()
opts.add_argument("--headless")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# 2) Go to the page and wait for at least one data row
url = "https://www.spotrac.com/nhl/free-agents/_/year/2025/type/ufa"
driver.get(url)
WebDriverWait(driver, 15).until(
    EC.presence_of_element_located((By.CSS_SELECTOR, "table tbody tr"))
)

# 3) Find all tables, parse each, and pick the one with data
df = None
tables = driver.find_elements(By.TAG_NAME, "table")
for tbl in tables:
    html = tbl.get_attribute("outerHTML")
    candidate = pd.read_html(html)[0]
    if not candidate.empty:
        df = candidate
        break

driver.quit()

if df is None:
    raise RuntimeError("No non‐empty <table> found on the page.")

# 4) Save to CSV
save_folder = "./data"
os.makedirs(save_folder, exist_ok=True)
out_path = os.path.join(save_folder, "nhl_ufa_2025.csv")
df.to_csv(out_path, index=False)

print(f"✅ Saved {df.shape[0]} rows × {df.shape[1]} columns to {out_path}")
print(df.head())

✅ Saved 292 rows × 7 columns to ./data\nhl_ufa_2025.csv
        Player (292) Pos   Age  YOE Prev Team     Prev AAV Type
0       John Tavares   C  34.6   14       TOR  $11,000,000  UFA
1    Mitchell Marner  RW  27.9    7       TOR  $10,903,000  UFA
2         Jamie Benn  LW  35.8   15       DAL   $9,500,000  UFA
3  Nicklas Backstrom   C  37.3   16       WAS   $9,200,000  UFA
4        Brent Burns   D  40.1   20       CAR   $8,000,000  UFA
